In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip3 install --upgrade ipython-autotime
%load_ext autotime
!pip3 install sentence_transformers=='2.2.2'

!pip3 install emoji=='2.2.0'

import pandas as pd
import numpy as np
# import pickle5 as pickle
import re
from nltk.stem import WordNetLemmatizer
import nltk
from tqdm.notebook import tqdm
tqdm.pandas()
import requests

from termcolor import colored
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

nb_workers=10

import os
from datetime import datetime
import pickle
import ast
import re
import emoji
from sentence_transformers import SentenceTransformer

lemmatizer = WordNetLemmatizer()
# punctuations = """!"$%()*+,*/:#»«'";“<=>?[\]^`”{.}~"""
punctuations = """!"$%()*+,*/:»«";“<=>?[\]^`”|{.}~"""
def clean_text_local(row):
#     row = re.sub(r'[^\x00-\x7F]+', r' \g<0> ',row)
    row = str(row)
    row = ''.join(' ' + char if emoji.is_emoji(char) else char for char in row)
    row = emoji.demojize(row)
    
#     row = re.sub('\+|>','',row).replace('<','\\').encode().decode('unicode-escape')
#     row=re.sub('\x0c',' ',row)
#     row=re.sub('\u200c',' ',row)
    # row=re.sub('\n\n',' . ',row)
    # row=re.sub('\\n',' ',row)
#     row=re.sub('\\\\n',' ',row)

    row=re.sub('\\\\n',' ',row)
#     row=re.sub('\n',' ',row)
    
    row=re.sub("-?NEWLINE_TOKEN", " ",row)
    row=re.sub("TAB_TOKEN", " ",row)
    row=re.sub("Alternate option=", "",row)
    row=re.sub('RT',' ',row)
    
    row=re.sub("@[A-Za-z0-9_]+"," ",row)
#     row=re.sub("#[A-Za-z0-9_]+"," ",row)
    row=re.sub("http\S+|www.\S+"," ",row)
    row=re.sub("\["," ",row)
    row=re.sub("\]"," ",row)
    row=re.sub("\("," ",row)
    row=re.sub("\)"," ",row)
    row=re.sub("^[A-Za-z]+: ","",row)
    # row = row.lower()

#     row = re.sub(r'<.*?>', '', row)
    row = re.sub("&amp","&",row)
    
#     row=re.sub("\d","",row)
#     row=re.sub(r"\b\d+\b|\b(?![i])[a-z]\b","",row) # removes every isolated number or char except i
    # row = ' '.join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(row)])
    # row=re.sub('-',' ',row)
    # row=re.sub('_',' ',row)
    # row=re.sub('nan','',row)
#     row=re.sub(':',' ',row)
#     no_punct = ""
#     for char in row:
#         if char not in punctuations:
#             no_punct = no_punct + char
    row = row.strip()
    row = re.sub('\s+',' ',row)
    return row
    # return row


model_st1 = SentenceTransformer('all-mpnet-base-v2') # version ='1'
print('loaded all-mpnet-base-v2', flush=True)

def embed(model, model_type, sentences):
    """
    wrapper function for generating message embeddings
    """
    if model_type == 'use':
        embeddings = model(list(sentences))
    elif model_type == 'sentence transformer':
        embeddings = model.encode(list(sentences),show_progress_bar=True)

    return embeddings

def split_embed(model_, model_type_, all_sentences_, n_partitions_=20):
    all_sentences_ = np.array_split(all_sentences_, indices_or_sections = n_partitions_)
    for i, new_sentences in enumerate(all_sentences_):
        if i==0:
            # print(colored('\nbeginning section {}'.format(i+1),'yellow'),flush=True)
            all_embeddings = embed(model_, model_type_, new_sentences)
        if i>0:
            # print(colored('\nbeginning section {}'.format(i+1),'yellow'),flush=True)
            new_embeddings = embed(model_, model_type_, new_sentences)
            all_embeddings = np.concatenate((all_embeddings,new_embeddings),axis=0)
    return all_embeddings

def vector_to_column(df,vectors,column_name):
    df[column_name]=0
    df[column_name]=df[column_name].astype(object)
    df.reset_index(drop=True,inplace=True)
    indices=df.index
    for i in indices:
        df.at[i, column_name]=vectors[i]
    return df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=8cf9b190f938f84f72a3b944639a41d1722a8145292b5814570866b31a0e2103
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd

loaded all-mpnet-base-v2
time: 41.2 s (started: 2023-05-22 19:44:30 +00:00)


In [ ]:
path_x = '.../Datasets/kialo_popular_tabular_format'
data = pd.read_csv(path_x)
data

time: 4.69 s (started: 2023-05-19 20:14:44 +00:00)


<ipython-input-3-ff10addfd9c8>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path_x)


In [ ]:
len(set(data[data['stance']=='Error']['post_id']))

20

time: 85.8 ms (started: 2023-05-19 20:20:03 +00:00)


In [ ]:
data['text_vectorized'] = data['text'].progress_apply(lambda x: clean_text_local(x))

  0%|          | 0/333617 [00:00<?, ?it/s]

time: 51.1 s (started: 2023-05-19 20:17:11 +00:00)


In [ ]:
data

,text_unclean,text,text_index,stance,discussion_id,discussion_name,file_name,stance_to_root,distance_to_root,text_vectorized,all-mpnet-base-v2,language,tags_list,text_translated
0,1. Blueprints for 3D-printed firearms should b...,Blueprints for 3D-printed firearms should be p...,1.,Topic,17593,3D Printer and Guns: Should Blueprints of 3D-P...,3d-printer-and-guns-should-blueprints-of-3d-pr...,Topic,0,Blueprints for 3D-printed firearms should be p...,"[0.006507655, 0.030841595, 0.009240107, 0.0324...",en,"gun,government,science",Blueprints for 3D-printed firearms should be p...
1,1.1. Pro: 3D Printing Makes Illegal Proliferat...,3D Printing Makes Illegal Proliferation of Gun...,1.1.,Pro,17593,3D Printer and Guns: Should Blueprints of 3D-P...,3d-printer-and-guns-should-blueprints-of-3d-pr...,Pro,1,3D Printing Makes Illegal Proliferation of Gun...,"[0.0054019117, 0.044243246, 0.002241348, -0.00...",en,"gun,government,science",3D Printing Makes Illegal Proliferation of Gun...
2,1.1.1. Con: It is [notoriously](https://money....,It is [notoriously](https://money.cnn.com/2015...,1.1.1.,Con,17593,3D Printer and Guns: Should Blueprints of 3D-P...,3d-printer-and-guns-should-blueprints-of-3d-pr...,Con,2,It is notoriously easy to buy guns in America....,"[-0.00086407014, 0.052978545, -0.007538742, -0...",en,"gun,government,science",It is [notoriously](https://money.cnn.com/2015...
3,1.1.2. Pro: With 3D Printers even indidividual...,With 3D Printers even indidividuals who wouldn...,1.1.2.,Pro,17593,3D Printer and Guns: Should Blueprints of 3D-P...,3d-printer-and-guns-should-blueprints-of-3d-pr...,Pro,2,With 3D Printers even indidividuals who wouldn...,"[-0.019681752, 0.10858277, 0.008730578, 0.0204...",en,"gun,government,science",With 3D Printers even indidividuals who wouldn...
4,1.2. Pro: The opposition to 3D guns is based o...,The opposition to 3D guns is based on the exis...,1.2.,Pro,17593,3D Printer and Guns: Should Blueprints of 3D-P...,3d-printer-and-guns-should-blueprints-of-3d-pr...,Pro,1,The opposition to 3D guns is based on the exis...,"[0.04648163, 0.026524125, 0.011560283, 0.02954...",en,"gun,government,science",The opposition to 3D guns is based on the exis...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357825,1.4.3. Con: Negative consequences of acknowled...,Negative consequences of acknowledging the tru...,1.4.3.,Con,6695,Does One Vote Make A Difference?,does-one-vote-make-a-difference-6695.txt,Pro,2,Negative consequences of acknowledging the tru...,"[-0.014781347, 0.038861167, 0.007119923, -0.00...",en,"elections,society,truth,politics",Negative consequences of acknowledging the tru...
357826,1.4.3.1. Con: It does if the factors in play r...,It does if the factors in play requires the mi...,1.4.3.1.,Con,6695,Does One Vote Make A Difference?,does-one-vote-make-a-difference-6695.txt,Con,3,It does if the factors in play requires the mi...,"[-0.042287186, 0.048748177, -0.0048393863, -0....",en,"elections,society,truth,politics",It does if the factors in play requires the mi...
357827,1.5. Con: At the margin one vote makes a diffe...,At the margin one vote makes a difference.,1.5.,Con,6695,Does One Vote Make A Difference?,does-one-vote-make-a-difference-6695.txt,Con,1,At the margin one vote makes a difference.,"[-0.03536318, 0.027356157, -0.0057575637, -0.0...",en,"elections,society,truth,politics",At the margin one vote makes a difference.
357828,1.5.1. Pro: In small constituencies an individ...,In small constituencies an individual vote mat...,1.5.1.,Pro,6695,Does One Vote Make A Difference?,does-one-vote-make-a-difference-6695.txt,Con,2,In small constituencies an individual vote mat...,"[-0.03212113, 0.039208263, -0.015957607, 0.005...",en,"elections,society,truth,politics",In small constituencies an individual vote mat...


time: 78.4 ms (started: 2023-04-26 07:47:53 +00:00)


In [ ]:
# generate embeddings for each model
embeddings_st1 = np.asarray(embed(model_st1, 'sentence transformer', data['text_vectorized']))
data = vector_to_column(df=data,vectors=embeddings_st1,column_name='all-mpnet-base-v2')
data

Batches:   0%|          | 0/10426 [00:00<?, ?it/s]

,text_unclean,text,text_index,stance,post_id,discussion_name,file_name,stance_to_root,distance_to_root,text_vectorized,all-mpnet-base-v2
0,1. Thesis:\n Online degrees should be valued e...,Online degrees should be valued equally to tra...,1.,Topic,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Topic,0,Online degrees should be valued equally to tra...,"[-0.015155796, 0.056269087, -0.032680623, -0.0..."
1,1.1. Con:\n Online college lacks interpersonal...,Online college lacks interpersonal skill devel...,1.1.,Con,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Con,1,Online college lacks interpersonal skill devel...,"[0.05471505, 0.04656394, -0.023736352, -0.0189..."
2,1.1.1. Con:\n Students can take charge of thei...,Students can take charge of their social lives...,1.1.1.,Con,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Pro,2,Students can take charge of their social lives...,"[0.08769956, -0.024991386, -0.018242788, 0.015..."
3,1.1.1.1. Pro:\n Without the extra expenses and...,"Without the extra expenses and travel times, s...",1.1.1.1.,Pro,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Pro,3,"Without the extra expenses and travel times, s...","[0.016782567, 0.024654506, 0.0035479276, 0.034..."
4,1.1.1.2. Pro:\n If students can make friends o...,If students can make friends outside of school...,1.1.1.2.,Pro,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Pro,3,If students can make friends outside of school...,"[0.058317453, 0.03630396, 0.016150597, 0.01076..."
...,...,...,...,...,...,...,...,...,...,...,...
333612,1.8.1.2. Con: This is not consistently applied...,This is not consistently applied now. Age disp...,1.8.1.2.,Con,5545,Should Incestuous Couples Have Marriage Equality?,should-incestuous-couples-have-marriage-equali...,Pro,3,This is not consistently applied now. Age disp...,"[0.010077229, 0.056066204, 0.050995924, 0.0520..."
333613,1.8.2. Pro: [Article 16](http://www.un.org/en/...,[Article 16](http://www.un.org/en/universal-de...,1.8.2.,Pro,5545,Should Incestuous Couples Have Marriage Equality?,should-incestuous-couples-have-marriage-equali...,Pro,2,Article 16 of the Universal Declaration of Hum...,"[-0.026209105, 0.02827814, 0.029886112, 0.0142..."
333614,1.8.3. Con: Most administrative marriage benef...,Most administrative marriage benefits would al...,1.8.3.,Con,5545,Should Incestuous Couples Have Marriage Equality?,should-incestuous-couples-have-marriage-equali...,Con,2,Most administrative marriage benefits would al...,"[-0.018854639, 0.099783204, -0.0062868176, 0.0..."
333615,1.8.4. Pro: Previously held beliefs - in parti...,Previously held beliefs - in particularly rega...,1.8.4.,Pro,5545,Should Incestuous Couples Have Marriage Equality?,should-incestuous-couples-have-marriage-equali...,Pro,2,Previously held beliefs - in particularly rega...,"[0.02312382, 0.12735082, 0.008172209, 0.002121..."


time: 14min 1s (started: 2023-05-19 20:21:25 +00:00)


In [ ]:
pickle.HIGHEST_PROTOCOL = 4
data.to_hdf(path_x+'.hdf','df')
data

<ipython-input-13-5ef590c1b36b>:2: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['text_unclean', 'text', 'text_index', 'stance', 'discussion_name',
       'file_name', 'stance_to_root', 'distance_to_root', 'text_vectorized',
       'all-mpnet-base-v2'],
      dtype='object')]

  data.to_hdf(path_x+'.hdf','df')


,text_unclean,text,text_index,stance,post_id,discussion_name,file_name,stance_to_root,distance_to_root,text_vectorized,all-mpnet-base-v2
0,1. Thesis:\n Online degrees should be valued e...,Online degrees should be valued equally to tra...,1.,Topic,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Topic,0,Online degrees should be valued equally to tra...,"[-0.015155796, 0.056269087, -0.032680623, -0.0..."
1,1.1. Con:\n Online college lacks interpersonal...,Online college lacks interpersonal skill devel...,1.1.,Con,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Con,1,Online college lacks interpersonal skill devel...,"[0.05471505, 0.04656394, -0.023736352, -0.0189..."
2,1.1.1. Con:\n Students can take charge of thei...,Students can take charge of their social lives...,1.1.1.,Con,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Pro,2,Students can take charge of their social lives...,"[0.08769956, -0.024991386, -0.018242788, 0.015..."
3,1.1.1.1. Pro:\n Without the extra expenses and...,"Without the extra expenses and travel times, s...",1.1.1.1.,Pro,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Pro,3,"Without the extra expenses and travel times, s...","[0.016782567, 0.024654506, 0.0035479276, 0.034..."
4,1.1.1.2. Pro:\n If students can make friends o...,If students can make friends outside of school...,1.1.1.2.,Pro,25243,Discussion Name:,should-online-degrees-be-valued-equally-to-tra...,Pro,3,If students can make friends outside of school...,"[0.058317453, 0.03630396, 0.016150597, 0.01076..."
...,...,...,...,...,...,...,...,...,...,...,...
333612,1.8.1.2. Con: This is not consistently applied...,This is not consistently applied now. Age disp...,1.8.1.2.,Con,5545,Should Incestuous Couples Have Marriage Equality?,should-incestuous-couples-have-marriage-equali...,Pro,3,This is not consistently applied now. Age disp...,"[0.010077229, 0.056066204, 0.050995924, 0.0520..."
333613,1.8.2. Pro: [Article 16](http://www.un.org/en/...,[Article 16](http://www.un.org/en/universal-de...,1.8.2.,Pro,5545,Should Incestuous Couples Have Marriage Equality?,should-incestuous-couples-have-marriage-equali...,Pro,2,Article 16 of the Universal Declaration of Hum...,"[-0.026209105, 0.02827814, 0.029886112, 0.0142..."
333614,1.8.3. Con: Most administrative marriage benef...,Most administrative marriage benefits would al...,1.8.3.,Con,5545,Should Incestuous Couples Have Marriage Equality?,should-incestuous-couples-have-marriage-equali...,Con,2,Most administrative marriage benefits would al...,"[-0.018854639, 0.099783204, -0.0062868176, 0.0..."
333615,1.8.4. Pro: Previously held beliefs - in parti...,Previously held beliefs - in particularly rega...,1.8.4.,Pro,5545,Should Incestuous Couples Have Marriage Equality?,should-incestuous-couples-have-marriage-equali...,Pro,2,Previously held beliefs - in particularly rega...,"[0.02312382, 0.12735082, 0.008172209, 0.002121..."


time: 11.3 s (started: 2023-05-19 20:35:26 +00:00)


In [3]:
data_paths = ['.../Datasets/Query-Results/Classification/economic_arguments_AI_gpt-3.5-turbo.xlsx',
             '.../Datasets/Query-Results/Classification/sociopolitical_arguments_AI_gpt-3.5-turbo.xlsx',
             '.../Datasets/Query-Results/Classification/economic_claims_Kialo.xlsx',
             '.../Datasets/Query-Results/Classification/sociopolitical_claims_Kialo.xlsx',
              '.../Datasets/Meta-Data/Kialo_MetaData_popular.xlsx']#,
              #'.../Datasets/Query-Results/Classification/all_arguments_AI_gpt-3.5-turbo.xlsx']
data_paths = ['.../Datasets/Query-Results/Classification/all_arguments_AI_proscons_gpt-3.5-turbo.xlsx']
# data = pd.read_excel(data_paths[4])

time: 562 µs (started: 2023-05-22 19:47:20 +00:00)


In [ ]:
data

,post_id,title,url,last_edit_date,found_where,initial_claim,tags_list,language,claim_translation,tags_list_translated
0,5637,Pro-Life vs Pro-Choice: Should Abortion be Legal?,https://www.kialo.com/pro-life-vs-pro-choice-s...,2023-03-15T01:58:59.677Z,https://www.kialo.com/explore/popular,Pregnant people should have the right to choos...,"Feminism,Abortion,Health,Religion,Women,Law",en,Pregnant people should have the right to choos...,"feminism,abortion,health,religion,women,law"
1,9563,Should the government provide funding for arts...,https://www.kialo.com/should-the-government-pr...,2023-02-25T19:29:52.672Z,https://www.kialo.com/explore/popular,The government should provide funding for arts...,"Art,Government,Economics,Education",en,The government should provide funding for arts...,"art,government,economics,education"
2,9226,Global overpopulation is a myth.,https://www.kialo.com/global-overpopulation-is...,2023-03-15T12:44:25.910Z,https://www.kialo.com/explore/popular,Global overpopulation is a myth.,"Society,Humans,Earth,Environment,ClimateChange",en,Global overpopulation is a myth.,"society,humans,earth,environment,climatechange"
3,1266,Gender Neutral Bathrooms: Should They be Stand...,https://www.kialo.com/gender-neutral-bathrooms...,2023-03-13T07:21:27.121Z,https://www.kialo.com/explore/popular,All public bathrooms should be gender neutral.,"Gender,Feminism,Transgender,Sex,Equality",en,All public bathrooms should be gender neutral.,"gender,feminism,transgender,sex,equality"
4,9647,Should homeschooling be banned?,https://www.kialo.com/should-homeschooling-be-...,2023-03-14T21:51:02.397Z,https://www.kialo.com/explore/popular,Homeschooling should be banned.,"Education,Policy,USA,Law,School,America",en,Homeschooling should be banned.,"education,policy,usa,law,school,america"
...,...,...,...,...,...,...,...,...,...,...
2821,60719,Культура отмены - путь к прогрессу,https://www.kialo.com/%D0%BA%D1%83%D0%BB%D1%8C...,2022-11-30T10:22:38.173Z,https://www.kialo.com/explore/popular,Культура отмены - путь к прогрессу,Культураотмены,ru,Cancellation culture is the way to progress,culture of cancellation
2822,61035,Нужно ли вводить 4-дневную рабочую неделю?,https://www.kialo.com/%D0%BD%D1%83%D0%B6%D0%BD...,2022-12-19T17:06:26.568Z,https://www.kialo.com/explore/popular,Насколько целесообразна идея ввести 4-х дневну...,Работа,ru,How expedient is the idea to introduce a 4-day...,work
2823,61059,A private money system with real bills and gol...,https://www.kialo.com/a-private-money-system-w...,2023-01-01T23:35:10.607Z,https://www.kialo.com/explore/popular,A private money system with real bills and gol...,Money,en,A private money system with real bills and gol...,money
2824,61106,Веганство:за и против,https://www.kialo.com/%D0%B2%D0%B5%D0%B3%D0%B0...,2022-12-29T05:30:37.694Z,https://www.kialo.com/explore/popular,Веганство:за и против,Веганство,mk,Veganism: pros and cons,veganism


time: 11.6 ms (started: 2023-05-17 18:00:02 +00:00)


In [4]:
for data_path in data_paths:
  data = pd.read_excel(data_path)
  try:
    text_column = 'arguments_list'
    data['text_vectorized'] = data[text_column].progress_apply(lambda x: clean_text_local(x))
  except:
    text_column = 'claim_translation'
    data['text_vectorized'] = data[text_column].progress_apply(lambda x: clean_text_local(x))

  embeddings_st1 = np.asarray(embed(model_st1, 'sentence transformer', data['text_vectorized']))
  data = vector_to_column(df=data,vectors=embeddings_st1,column_name='all-mpnet-base-v2')
  print(data)
  pickle.HIGHEST_PROTOCOL = 4
  data.to_hdf('.../Datasets/Data - Arguments - Human vs AI/'+re.sub('.xlsx$','',data_path.split('/')[-1])+'.hdf','df')

  0%|          | 0/21357 [00:00<?, ?it/s]

Batches:   0%|          | 0/668 [00:00<?, ?it/s]

       post_id                                                url  \
0         5637  https://www.kialo.com/pro-life-vs-pro-choice-s...   
1         5637  https://www.kialo.com/pro-life-vs-pro-choice-s...   
2         5637  https://www.kialo.com/pro-life-vs-pro-choice-s...   
3         5637  https://www.kialo.com/pro-life-vs-pro-choice-s...   
4         5637  https://www.kialo.com/pro-life-vs-pro-choice-s...   
...        ...                                                ...   
21352    61071  https://www.kialo.com/debatiendo-la-escuela-%F...   
21353    61071  https://www.kialo.com/debatiendo-la-escuela-%F...   
21354    61071  https://www.kialo.com/debatiendo-la-escuela-%F...   
21355    61071  https://www.kialo.com/debatiendo-la-escuela-%F...   
21356    61071  https://www.kialo.com/debatiendo-la-escuela-%F...   

                                           initial_claim          model  \
0      Pregnant people should have the right to choos...  gpt-3.5-turbo   
1      Pregnant peopl

<ipython-input-4-b97793d0e545>:14: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['url', 'initial_claim', 'model', 'query', 'answer', 'language',
       'answer_translated', 'tags_list_translated', 'subject_np',
       'arguments_list', 'stance_given', 'text_vectorized',
       'all-mpnet-base-v2'],
      dtype='object')]

  data.to_hdf('/content/drive/MyDrive/Kialo vs ChatBots/Data - Arguments - Human vs AI/'+re.sub('.xlsx$','',data_path.split('/')[-1])+'.hdf','df')


time: 56.4 s (started: 2023-05-22 19:47:47 +00:00)
